# Risk Analytics

Risk analytics is the practice of using data, statistical models, and simulations to identify, measure, and manage potential losses in uncertain environments. It helps organizations anticipate different risk scenarios by quantifying how likely and severe those risks might be, enabling smarter decision-making.

## Value-at-Risk (VaR)

There are several methods to evaluate the risk of an individual stock or a portfolio, such as variance and standard deviation of returns. However, these measures do not take into account the probability distribution of potential losses.

Many risk managers prefer a more intuitive and probabilistic measure called Value at Risk (VaR). It is one of the most widely used metrics for assessing the risk of a financial position or a portfolio of financial instruments. VaR is defined as:

`The maximum expected loss over a given time horizon, at a specified confidence level, under normal market conditions.`

For example, if the 1-day 95% VaR of a portfolio is $100, this means that there is a 95% confidence that the portfolio will not lose more than $100 over the next day.

There are several methods to calculate VaR, including:

* Parametric (Variance-Covariance)

* Historical Simulation

* Monte Carlo Simulation

* Modified VaR (e.g., using Cornish-Fisher expansion)



**Import Libraries**

In [ ]:
# Import libraries
import sys
import os
sys.path.append(os.path.abspath(".."))

import numpy as np
import pandas as pd
import quantmod.charts
from utils import query_all_stocks
from pprint import pprint
from numpy.linalg import multi_dot
from collections import OrderedDict

from scipy import stats
from tabulate import tabulate

# Set precision
pd.set_option('display.precision', 4)

## Data Retrieval

We will retrieve price data for selected stocks from our database to build our portfolio.

In [ ]:
# Query stock data from database
df = query_all_stocks()
assets = sorted(['ICICIBANK', 'ITC', 'RELIANCE', 'TCS', 'ASIANPAINT'])
df = df[assets]
returns = df.pct_change().dropna()

# Select a single stock
stockreturn = returns['ICICIBANK']

# Calculate mean and standard deviation 
mean = stockreturn.mean()
stdev = stockreturn.std()

returns.head()

### Parametric VaR

The Variance-Covariance method is a parametric approach that assumes asset returns are normally distributed. Under this assumption, risk can be estimated using just the mean ($\mu$) and standard deviation ($\sigma$) of returns.

The formula for VaR under this method is:

$$VaR=position×(μ−z⋅σ)$$

Common Z-Scores for Confidence Levels:

| Confidence Level  |Z-score         | Value At Risk     
| :-                |:---------      |:---------            
|`90%`              |1.28             |$\mu$ - $1.28$ * $\sigma$ 
|`95%`              |1.64             |$\mu$ - $1.64$ * $\sigma$             
|`99%`              |2.33             |$\mu$ - $2.33$ * $\sigma$             

where, $\mu$ is the mean return, $\sigma$ is the volatility and $z$ is the number of standard deviation from the mean.

Note: These z-values come from the standard normal distribution.

The parametric VaR method is fast and easy to compute, but its accuracy heavily depends on the normality assumption, which may underestimate risk in extreme market conditions.

In [ ]:
# Portfolio or position value
position_value = 1

# Calculate VaR at difference confidence level
VaR_90 = stats.norm.ppf(0.10, loc=mean, scale=stdev)
VaR_95 = stats.norm.ppf(0.05, loc=mean, scale=stdev)
VaR_99 = stats.norm.ppf(0.01, loc=mean, scale=stdev)

In [ ]:
# Ouput results in tabular format
table = [['90%', VaR_90],['95%', VaR_95],['99%', VaR_99]]
header = ['Confidence Level', 'Value At Risk']
print(tabulate(table,headers=header))

### Historical VaR

In practice, asset returns often deviate from the normal distribution — exhibiting skewness, kurtosis, and fat tails. To account for this, the Historical VaR method offers a non-parametric alternative that does not assume any specific distribution for returns.

Instead, it uses actual historical returns to estimate risk. The steps are:

* Collect a time series of historical returns.

* Sort the returns in ascending order.

* Identify the return at the desired percentile threshold corresponding to the confidence level.

For example, the 5th percentile (i.e., the return at the bottom 5%) corresponds to a 95% confidence level. This return is interpreted as the maximum expected loss under normal historical conditions with the chosen confidence level. If the 1-day Historical VaR at the 95% confidence level is -2.3%, it means, in 95% of past trading days, the portfolio did not lose more than 2.3% in a single day.

In [ ]:
# Use quantile function for Historical VaR
hVaR_90 = np.percentile(stockreturn, 10)
hVaR_95 = np.percentile(stockreturn, 5)
hVaR_99 = np.percentile(stockreturn, 1)

In [ ]:
# Ouput results in tabular format
htable = [['90%', hVaR_90],['95%', hVaR_95],['99%', hVaR_99]]
print(tabulate(htable,headers=header))


### Monte Carlo VaR

The Monte Carlo simulation approach is similar to historical simulation in that it uses many simulated possible future returns to estimate risk. However, unlike historical VaR, which uses actual past returns, Monte Carlo VaR assumes that returns follow a specified probability distribution (commonly normal distribution).

Using the mean and standard deviation calculated from historical returns, the Monte Carlo method generates n random simulated returns from the assumed distribution. These simulated returns are then sorted in ascending order to estimate the maximum potential loss at a given confidence level.

This approach allows flexibility to model complex return distributions and risk factors but depends on the accuracy of the assumed distribution.

In [ ]:
# Set seed for reproducibility
np.random.seed(42)

# Number of simulations
n_sims = 10000

# Simulate returns assuming normal distribution with given mean and stdev
sim_returns = np.random.normal(mean, stdev, n_sims)

# Calculate Monte Carlo VaR at different confidence levels using percentiles
MCVaR_90 = np.percentile(sim_returns, 10)  
MCVaR_95 = np.percentile(sim_returns, 5)  
MCVaR_99 = np.percentile(sim_returns, 1)  

In [ ]:
# Ouput results in tabular format
mctable = [['90%', MCVaR_90],['95%', MCVaR_95],['99%', MCVaR_99]]
print(tabulate(mctable,headers=header))

### Normality Test

In the Parametric VaR method, we assume that returns are normally distributed. However, in practice, financial return distributions often show skewness, kurtosis, and fat tails, which deviate from the normal distribution.

Before relying on a parametric VaR approach, it's important to check whether this normality assumption holds for the asset or portfolio returns.

#### Shapiro-Wilk Test

The Shapiro-Wilk test is a statistical test used to assess normality of a dataset. It tests the null hypothesis that the data was drawn from a normal distribution.

* _Null Hypothesis ($H_0$): Data is normally distributed_

* _Alternative Hypothesis ($H_1$): Data is not normally distributed_    

* _A p-value < 0.05 suggests that the distribution significantly deviates from normality._

In [ ]:
# Perform the Shapiro-Wilk test
stats.shapiro(stockreturn)

Given that the p-value is extremely small (< 0.05), we reject the null hypothesis. This provides strong statistical evidence that the sample of daily returns does not come from a normal distribution. This result is not surprising. Financial returns are known to exhibit fat tails, Skewness, and Volatility clustering. Because this dataset reflects empirical financial returns, it's expected to deviate from the idealized bell curve assumed in parametric models.

Since the normality assumption is violated, we should not rely solely on Parametric VaR, which assumes returns are normally distributed. This assumption, when incorrect, can lead to underestimation of tail risk and inaccurate risk metrics. Instead, more robust alternatives should be considered, such as Historical VaR, Monte Carlo with non-normal distributions, or Modified VaR (e.g., Cornish-Fisher) if you want to adjust for skew and kurtosis. These methods provide a more accurate representation of financial risk under non-normal conditions and are better suited for empirical return distributions.

#### Anderson-Darling Test

As an alternative to the Shapiro-Wilk test, we can use the Anderson-Darling test, a goodness-of-fit test that evaluates how well a given dataset fits a specified probability distribution. This test is particularly sensitive to deviations in the tails of the distribution, which makes it especially useful in finance, where extreme values (tail risk) matter. It is most commonly used to assess whether data follows a normal distribution, although it can be used to test other distributions as well.

* _Null Hypothesis ($H_0$): The data follows the specified distribution (e.g., normal)._

* _The test returns a statistic and critical values for various significance levels (15%, 10%, 5%, 2.5%, 1%)._

* _If the test statistic is greater than the critical value for your chosen significance level, you reject the null hypothesis._



In [ ]:
# Run the Anderson-Darling test for normality
stats.anderson(stockreturn, dist='norm')

Since the test statistic (2.9662) is greater than all critical values (even at the 1% level), we reject the null hypothesis. There is strong evidence that the return distribution significantly deviates from normality. This result is consistent with what we expect in real-world financial data.

**Plot Histogram**

We will now plot histogram to visualize the returns distribution

In [ ]:
# Plot histogram of daily returns
returns.iplot(kind="histogram", overlap=False, showlegend=True)

### Modified VaR 

The standard normal distribution is defined by four key statistical moments: Mean ($\mu$): 0, Variance ($\sigma^2$): 1, Skewness ($s$): 0 and Kurtosis ($k$): 3

However, in real-world financial returns, the skewness and excess kurtosis are often non-zero. This violates the assumptions of normality, making standard Parametric VaR insufficient. To address this, the Modified VaR (also known as Cornish-Fisher VaR) incorporates the third and fourth moments, skewness and kurtosis along with mean and standard deviation, to better estimate tail risk.

$$mVaR=position×(μ−t⋅σ)$$

Where, $$t = z + \frac{1}{6} (z^{2} - 1)s + \frac{1}{24}(z^{3} - 3z)k - \frac{1}{36}(2z^{3} - 5z)s^{2}$$

$\mu$ is the mean return, $\sigma$ is the standard deviation of returns, $s$ is skewness, $k$ is excess kurtosis (i.e., kurtosis − 3), $z$ is the z-score for the desired confidence level (e.g., 1.64 for 95%) and $t$ is the Cornish-Fisher adjusted quantile.

Modified VaR adjusts for asymmetry (skewness) and fat tails (excess kurtosis), making it a more robust estimator than the standard parametric VaR. It effectively bridges the gap between the simplicity of parametric VaR and the computational intensity of full Monte Carlo simulations.

In [ ]:
dist = OrderedDict({
    'Mean': np.mean(stockreturn),
    'StdDev': np.std(stockreturn),
    'Skew': stats.skew(stockreturn),
    'Kurtosis': stats.kurtosis(stockreturn)
})

pprint(dist)

In [ ]:
mean = np.mean(stockreturn)
stdev = np.std(stockreturn)
s = stats.skew(stockreturn)
k = stats.kurtosis(stockreturn)
z = abs(stats.norm.ppf(0.01))

t = z + (1/6)*(z**2 - 1)*s + (1/24)*(z**3 - 3*z)*k - (1/36)*(2*z**3 - 5*z)*s**2

mVaR_99 = -(mean - t * stdev) 
print(f"Modified VaR (99%): {mVaR_99}")

## Expected Shortfall (Conditional Value at Risk, CVaR)

Value at Risk (VaR) is a popular risk metric, but it can underestimate the risk if the return distribution has fat tails or overestimate the risk if the tails are thinner than assumed.

Expected Shortfall or Conditional Value at Risk (CVaR) provides a more coherent risk measure by estimating the expected loss conditional on losses exceeding the VaR threshold. In other words, it calculates the average loss in the worst 1- 𝑐𝑙% of cases.

Assuming we have 𝑛 return observations, CVaR at confidence level 𝑐l is defined as:

$$CVaR = \frac{1}{n} \sum_{i=1}^{n} R_i \cdot \mathbf{1}_{\{ R_i \leq VaR_{cl} \}}$$

where,
* $R_i$ are the returns, 
* $VaR_{cl}$ is the VaR threshold at confidence level $cl$,
* $\mathbf{1}_{\{.\}}$ is the indicator function that is 1 if the condition holds, 0 otherwise.



Put simply, CVaR is the average return of all losses worse than the VaR threshold, making it a more informative risk measure in the presence of fat tails or skewness.

In [ ]:
# Calculate CVar
CVaR_90 = -stockreturn[stockreturn <= hVaR_90].mean()
CVaR_95 = -stockreturn[stockreturn <= hVaR_95].mean()
CVaR_99 = -stockreturn[stockreturn <= hVaR_99].mean()

In [ ]:
# Ouput results in tabular format
ctable = [['90%', CVaR_90],['95%', CVaR_95],['99%', CVaR_99] ]
cheader = ['Confidence Level', 'Conditional Value At Risk']
print(tabulate(ctable,headers=cheader))

## Portfolio Value at Risk

When assessing portfolio-level risk, it's important to account not only for the individual returns and volatilities of the assets but also for their correlations with each other.

In this section, we focus on estimating the VaR of a minimum variance portfolio constructed from the previous section. A minimum variance portfolio aims to achieve the lowest possible volatility by optimally weighting assets based on their covariance structure. Once we determine the optimal weights, we can compute the portfolio's overall volatility and subsequently derive its VaR.

This approach provides a more accurate and diversified measure of risk compared to evaluating individual asset VaRs in isolation.



In [ ]:
# Weights from Minimum Variance Portfolio 
# assets = ['ASIANPAINT', 'ICICIBANK', 'ITC', 'RELIANCE', 'TCS']
wts = np.array([0.25951357, 0.05657516, 0.29447842, 0.0772431 , 0.31218975])

# Portfolio mean returns and volatility
port_mean = wts.T @ returns.mean()
port_stdev = np.sqrt(multi_dot([wts.T, returns.cov(), wts]))

pVaR = stats.norm.ppf(0.01, port_mean, port_stdev)

print(f"Mean: {port_mean}, Stdev: {port_stdev}, pVaR: {pVaR}")

## Quantmod Risk Module

The quantmod risk module offers a comprehensive set of tools to assess the risk associated with financial portfolios. It includes key risk metrics such as Value at Risk (VaR) and Conditional Value at Risk (CVaR), along with functionality for VaR backtesting to evaluate the accuracy of risk forecasts. These tools are essential for quantifying potential losses under normal and extreme market conditions, making them valuable for portfolio risk management and compliance. 

Key features include:

* RiskInputs: A structured input class to standardize asset returns, confidence levels, and time horizons for risk measurement.

* ValueAtRisk: Computes the portfolio’s VaR using multiple models.

* ConditionalVaR (CVaR): Estimates the expected loss in the tail.

* VarBacktester: Facilitates VaR backtesting by comparing predicted VaR values to realized returns, counting breaches and supporting statistical evaluation of model accuracy.

In [ ]:
# Import Quantmod Risk Metrics
from quantmod.risk import RiskInputs, VaRMetrics, VaRAnalyzer

### Single Stock Risk Analysis

In [ ]:
single_stock_risk_metrics = VaRMetrics(
    RiskInputs(
        confidence_level=0.99,
        lookback_period=252,
        num_simulations=10000,
        portfolio_weights=None,
        portfolio_returns=stockreturn.to_frame(),
        is_single_stock=True,
    )
)

In [ ]:
print(
    f"\nSingle Stock VaR and CVaR for the given confidence level"
)
print(f"Parametric VaR : {single_stock_risk_metrics.parametric_var:.4f}")
print(f"Historical VaR : {single_stock_risk_metrics.historical_var:.4f}")
print(f"Monte Carlo VaR : {single_stock_risk_metrics.monte_carlo_var:.4f}")
print(f"Expected Shortfall : {single_stock_risk_metrics.expected_shortfall:.4f}")

### Portfolio Risk Analysis

In [ ]:
# Portfolio
portfolio_risk_metrics = VaRMetrics(
    RiskInputs(
        confidence_level=0.99,
        lookback_period=252,
        num_simulations=10000,
        portfolio_weights=[
            0.25951357, 
            0.05657516, 
            0.29447842, 
            0.0772431 , 
            0.31218975            
        ],
        portfolio_returns=returns,
        is_single_stock=False,
    )
)

In [ ]:
# Print results
print("Portfolio VaR and CVaR for the given confidence level")
print(f"Parametric VaR : {portfolio_risk_metrics.parametric_var:.4f}")
print(f"Historical VaR : {portfolio_risk_metrics.historical_var:.4f}")
print(f"Monte Carlo VaR : {portfolio_risk_metrics.monte_carlo_var:.4f}")
print(f"Expected Shortfall : {portfolio_risk_metrics.expected_shortfall:.4f}")

### VaR Backtesting Analysis

VaR backtesting is used to assess how well a VaR model predicts actual losses. It involves comparing the predicted VaR values to the realized portfolio returns to identify instances where losses exceed the expected threshold, known as VaR breaches.

The Quantmod risk module provides a convenient framework for performing VaR backtests. It counts the number of breaches and evaluates whether they are statistically consistent with the model’s confidence level. This helps determine whether the VaR model underestimates or overestimates risk, making it a critical component of risk model validation and compliance.

In [ ]:
# VaR Backtest
backtest_results = VaRAnalyzer(
    inputs=RiskInputs(
        portfolio_returns=stockreturn.to_frame(),
        confidence_level=0.99,
        lookback_period=252,
    )
).run
print(f" Backtested VaR Results: \n {backtest_results}")

---
[Kannan Singaravelu](https://www.linkedin.com/in/kannansi) | Refer [Quantmod](https://kannansingaravelu.com/quantmod/) for more information.